In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timezone

In [2]:
from sqlalchemy import create_engine, text, bindparam
from sqlalchemy.dialects.postgresql import UUID, ARRAY
from dotenv import load_dotenv
from surprise import Dataset, Reader, SVD, SVDpp, accuracy
from surprise.model_selection import train_test_split

In [3]:
load_dotenv()

True

In [4]:
# --- Environment & DB connection ------------------------------------------------
DATABASE_USERNAME = os.environ.get('DATABASE_USERNAME')
DATABASE_PASSWORD = os.environ.get('DATABASE_PASSWORD')
DATABASE_HOST = os.environ.get('DATABASE_HOST')
DATABASE_PORT = os.environ.get('DATABASE_PORT')
DATABASE_NAME = os.environ.get('DATABASE_NAME')

In [5]:
# Build Postgres DSN and create a SQLAlchemy engine
PG_DSN = f"postgresql+psycopg2://{DATABASE_USERNAME}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(PG_DSN, pool_pre_ping=True)

In [6]:
# --- Aggregation parameters (keep in sync with your MV if you use one) ----------
HALF_LIFE_DAYS = 30          # time-decay half-life in days (fresh events weigh more)
VIEW_DEBOUNCE_SECONDS = 30   # debounce window for 'view' events from same user on same yacht
LOOKBACK_DAYS = 365 * 2      # history window for events (e.g., last 2 years)
DAY_CAP = 20.0               # per-(user,yacht,day) cap to prevent single session flooding

-----------------------------
--- DATA PREPARATION --------
-----------------------------

In [7]:
# --- 1) Load base tables --------------------------------------------------------
# Users: only need id and role to keep 'lessee' events
df_users = pd.read_sql('SELECT id, role FROM users;', engine)
df_users.head()

,id,role
0,1a4f43f7-52e8-456d-8a96-34b704ba7eb6,lessee
1,1d7dbee7-2c6c-4d6a-b36c-4dc50a14b120,lessee
2,24e79705-1038-4d6f-b4ad-cda02fd6e3da,lessee
3,304d7f41-c837-43bc-af43-0c554cef88f1,lessee
4,4150a63e-ab37-4f8a-82ef-97aa373888ec,lessee


In [8]:
# Yachts: (owner_id used to drop self-interactions)
df_yachts = pd.read_sql('SELECT id as yacht_id, "userId" AS owner_id FROM yachts;', engine)
df_yachts.head()

,yacht_id,owner_id
0,e56888db-0317-42a3-a057-896daf3cd5ef,ff210a03-d01e-49a5-9050-284d1d94490a
1,e471618c-f35d-400d-912c-c217f4650a04,ff210a03-d01e-49a5-9050-284d1d94490a
2,9149396e-5737-4ae6-a29e-9129aafe9059,ff210a03-d01e-49a5-9050-284d1d94490a
3,87d38b20-5f94-49df-a405-711a8097408e,ff210a03-d01e-49a5-9050-284d1d94490a
4,6f5d2cca-51d9-4395-8c33-fd3aa8c6b07e,ff210a03-d01e-49a5-9050-284d1d94490a


In [9]:
# Events within the lookback window (parameterized to avoid string formatting in SQL)
events_query = text("""
    SELECT id,
           "userId"   AS user_id,
           "yachtId"  AS yacht_id,
           type,
           weight,
           "createdAt" AS ts
    FROM events
    WHERE "createdAt" >= NOW() - (:days || ' days')::interval
""")
df_events = pd.read_sql(events_query, engine, params={'days': str(LOOKBACK_DAYS)})

# Optional guard: if there are no events, stop early to avoid errors downstream
if df_events.empty:
    raise ValueError("No events found within the lookback window. Adjust LOOKBACK_DAYS or check data.")

df_events.head()

,id,user_id,yacht_id,type,weight,ts
0,6770dd7e-930a-41f3-92dd-aba9608e738d,d5977da5-42c5-4fa2-a44c-55bd047e835d,e56888db-0317-42a3-a057-896daf3cd5ef,view,2,2025-03-31 17:16:35.240000+00:00
1,bb131e30-bf37-425d-96bc-501e16416e97,23f09fda-ccba-4081-a482-b3d4d3e2d217,e471618c-f35d-400d-912c-c217f4650a04,view,2,2025-10-16 23:31:49.363000+00:00
2,79630cc9-6459-4ce3-bfe9-640cb6a8f96f,0eeb1f59-3484-4c4e-9cf6-42e5e877838f,87d38b20-5f94-49df-a405-711a8097408e,view,2,2025-06-22 14:51:18.974000+00:00
3,9e28bfe9-bf7c-4298-a63d-92888b21994e,23f09fda-ccba-4081-a482-b3d4d3e2d217,87d38b20-5f94-49df-a405-711a8097408e,view,2,2024-12-31 05:42:03.879000+00:00
4,201f8c43-1078-433a-937c-26c85ddcbc6d,0670dada-5602-449f-95a2-92d24c2c750a,5d09a41f-4f28-4396-afcd-0d308c25efec,view,2,2025-08-22 19:43:53.241000+00:00


In [12]:
df_users.shape

(104, 2)

In [13]:
df_yachts.shape

(2544, 2)

In [14]:
df_events.shape

(1499, 6)

In [15]:
# Optional guard: if there are no events, stop early to avoid errors downstream
if df_events.empty:
    raise ValueError("No events found within the lookback window. Adjust LOOKBACK_DAYS or check data.")
    
# --- 2) Keep only 'lessee' events ----------------------------------------------
# Join user role and filter to renter role (events from admins/owners are not demand signals)
df_events = df_events.merge(df_users, left_on="user_id", right_on="id", how="left", suffixes=("", "_u"))
df_events = df_events[df_events["role"] == "lessee"].drop(columns=["id_u", "role"])

# --- 2) Keep only 'lessee' events ----------------------------------------------
# Join user role and filter to renter role (events from admins/owners are not demand signals)
df_events = df_events.merge(df_users, left_on="user_id", right_on="id", how="left", suffixes=("", "_u"))
df_events = df_events[df_events["role"] == "lessee"].drop(columns=["id_u", "role"])

# --- 3) Remove interactions with user's own yachts ------------------------------
# (Owners interacting with their listings would bias the model)
df_events = df_events.merge(df_yachts, on="yacht_id", how="left")
df_events = df_events[df_events["owner_id"].astype(str) != df_events["user_id"].astype(str)]

# --- 4) Debounce frequent 'view' events ----------------------------------------
# Sort and drop near-duplicate 'view' events per (user, yacht) inside the debounce window
df_events = df_events.sort_values(["user_id", "yacht_id", "type", "ts"])
mask = ~(
    (df_events["type"] == "view") &
    (df_events.groupby(["user_id", "yacht_id", "type"])["ts"]
        .diff()
        .dt.total_seconds()
        .between(0, VIEW_DEBOUNCE_SECONDS, inclusive="both"))
)
df_events = df_events[mask]

# Optional guard: if everything got filtered out, stop early
if df_events.empty:
    raise ValueError("All events were filtered out by role/self-ownership/debounce. Check filters.")

In [16]:
df_events.shape

(1499, 7)

In [18]:
# --- 5) Time-decay weighting ----------------------------------------------------
# Compute exponentially decayed weight: eff_w = weight * exp(-days_ago / half_life)
now = pd.Timestamp.utcnow()
days_ago = (now - pd.to_datetime(df_events["ts"], utc=True)).dt.total_seconds() / 86400.0
decay = np.exp(-days_ago / HALF_LIFE_DAYS)
df_events["eff_w"] = df_events["weight"].astype(float) * decay

# --- 6) Daily cap to limit flood from single sessions ---------------------------
# Aggregate by day, cap the daily contribution, then sum across days
df_events["d"] = pd.to_datetime(df_events["ts"], utc=True).dt.floor("D")
df_events.head()

,id,user_id,yacht_id,type,weight,ts,owner_id,eff_w,d
495,e02d836d-2ecf-43a7-a231-aab7dfa92959,0176a5a3-e4de-4365-a131-b62f095d7e8a,0be7a625-065c-4875-a70e-632d6005c2a8,view,2,2025-10-03 05:42:25.899000+00:00,ff210a03-d01e-49a5-9050-284d1d94490a,0.830909,2025-10-03 00:00:00+00:00
1019,d15b0914-3122-493a-9a19-84508817e3a8,0176a5a3-e4de-4365-a131-b62f095d7e8a,184195fe-a387-49cb-ab63-c3d08de09fc0,view,2,2025-10-18 07:17:16.525000+00:00,ff210a03-d01e-49a5-9050-284d1d94490a,1.372949,2025-10-18 00:00:00+00:00
553,1a0ef70e-c902-44af-9751-e0da366a6509,0176a5a3-e4de-4365-a131-b62f095d7e8a,1e22d50f-0e8e-4df1-8cde-342135867737,view,2,2025-10-20 13:42:29.489000+00:00,ff210a03-d01e-49a5-9050-284d1d94490a,1.480744,2025-10-20 00:00:00+00:00
13,5814093b-d3f5-4b3e-a950-8d0786f6212b,0176a5a3-e4de-4365-a131-b62f095d7e8a,2daf28cb-4009-4060-89de-6cf026e9a296,wishlist,4,2025-10-10 08:21:56.013000+00:00,ff210a03-d01e-49a5-9050-284d1d94490a,2.106311,2025-10-10 00:00:00+00:00
374,90415064-e75f-4fad-b2c3-946c70945a08,0176a5a3-e4de-4365-a131-b62f095d7e8a,5740a809-5ff2-41c7-bee7-3f5eed723790,wishlist,4,2025-10-18 16:08:03.412000+00:00,ff210a03-d01e-49a5-9050-284d1d94490a,2.779844,2025-10-18 00:00:00+00:00


In [19]:
daily = (
    df_events
    .groupby(["user_id", "yacht_id", "d"], as_index=False)["eff_w"]
    .sum()
)
daily["day_sum"] = daily["eff_w"].clip(upper=DAY_CAP)

daily.head()

,user_id,yacht_id,d,eff_w,day_sum
0,0176a5a3-e4de-4365-a131-b62f095d7e8a,0be7a625-065c-4875-a70e-632d6005c2a8,2025-10-03 00:00:00+00:00,0.830909,0.830909
1,0176a5a3-e4de-4365-a131-b62f095d7e8a,184195fe-a387-49cb-ab63-c3d08de09fc0,2025-10-18 00:00:00+00:00,1.372949,1.372949
2,0176a5a3-e4de-4365-a131-b62f095d7e8a,1e22d50f-0e8e-4df1-8cde-342135867737,2025-10-20 00:00:00+00:00,1.480744,1.480744
3,0176a5a3-e4de-4365-a131-b62f095d7e8a,2daf28cb-4009-4060-89de-6cf026e9a296,2025-10-10 00:00:00+00:00,2.106311,2.106311
4,0176a5a3-e4de-4365-a131-b62f095d7e8a,5740a809-5ff2-41c7-bee7-3f5eed723790,2025-10-18 00:00:00+00:00,2.779844,2.779844


In [21]:
scores = (
    daily
    .groupby(["user_id", "yacht_id"], as_index=False)
    .agg(score=("day_sum", "sum"), latest_ts=("d", "max"))
)
print(scores.shape)
scores.head()


(1491, 4)


,user_id,yacht_id,score,latest_ts
0,0176a5a3-e4de-4365-a131-b62f095d7e8a,0be7a625-065c-4875-a70e-632d6005c2a8,0.830909,2025-10-03 00:00:00+00:00
1,0176a5a3-e4de-4365-a131-b62f095d7e8a,184195fe-a387-49cb-ab63-c3d08de09fc0,1.372949,2025-10-18 00:00:00+00:00
2,0176a5a3-e4de-4365-a131-b62f095d7e8a,1e22d50f-0e8e-4df1-8cde-342135867737,1.480744,2025-10-20 00:00:00+00:00
3,0176a5a3-e4de-4365-a131-b62f095d7e8a,2daf28cb-4009-4060-89de-6cf026e9a296,2.106311,2025-10-10 00:00:00+00:00
4,0176a5a3-e4de-4365-a131-b62f095d7e8a,5740a809-5ff2-41c7-bee7-3f5eed723790,2.779844,2025-10-18 00:00:00+00:00


In [23]:
# --- 7) Per-user min-max scaling to [1..5] -------------------------------------
# Convert implicit scores into pseudo-ratings for Surprise (stable range per user)
def scale_1_5(g: pd.DataFrame) -> pd.DataFrame:
    mn, mx = g["score"].min(), g["score"].max()
    if mx - mn < 1e-9:
        # If user has uniform scores, assign a neutral-but-positive rating
        g["rating"] = 4.0
    else:
        g["rating"] = 1 + 4 * (g["score"] - mn) / (mx - mn)
    return g

def ensure_str_ids(df):
    df["user_id"] = df["user_id"].astype(str)
    df["yacht_id"] = df["yacht_id"].astype(str)
    return df

In [25]:
ratings = scores.groupby("user_id", group_keys=False).apply(scale_1_5)
ratings = ratings[["user_id", "yacht_id", "rating", "latest_ts"]].reset_index(drop=True)
ratings = ensure_str_ids(ratings)
print("Number of ratings:\n", ratings.shape)
# print("Sample ratings:\n", ratings.head())
ratings.head()

Number of ratings:
 (1491, 4)


/var/folders/lj/mz0mblqx32ldyss3l0yjwz1r0000gp/T/ipykernel_79195/81667383.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ratings = scores.groupby("user_id", group_keys=False).apply(scale_1_5)


,user_id,yacht_id,rating,latest_ts
0,0176a5a3-e4de-4365-a131-b62f095d7e8a,0be7a625-065c-4875-a70e-632d6005c2a8,1.081099,2025-10-03 00:00:00+00:00
1,0176a5a3-e4de-4365-a131-b62f095d7e8a,184195fe-a387-49cb-ab63-c3d08de09fc0,2.171027,2025-10-18 00:00:00+00:00
2,0176a5a3-e4de-4365-a131-b62f095d7e8a,1e22d50f-0e8e-4df1-8cde-342135867737,2.387781,2025-10-20 00:00:00+00:00
3,0176a5a3-e4de-4365-a131-b62f095d7e8a,2daf28cb-4009-4060-89de-6cf026e9a296,3.645666,2025-10-10 00:00:00+00:00
4,0176a5a3-e4de-4365-a131-b62f095d7e8a,5740a809-5ff2-41c7-bee7-3f5eed723790,5.000000,2025-10-18 00:00:00+00:00


In [ ]:
# --- 8) Build Surprise dataset --------------------------------------------------
# Surprise expects explicit ratings with a known scale; we mapped to [1..5]
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[["user_id", "yacht_id", "rating"]], reader)

-----------------------------
--- TRAIN / TEST MODEL --------
-----------------------------

In [26]:
# Config
ART_DIR = os.getenv("ART_DIR", "./artifacts_surprise")
ALGO_NAME = os.getenv("ALGO", "SVD")            # 'SVD' or 'SVDpp'
FACTORS   = int(os.getenv("FACTORS", "64"))
EPOCHS    = int(os.getenv("EPOCHS", "40"))
LR_ALL    = float(os.getenv("LR_ALL", "0.005"))
REG_ALL   = float(os.getenv("REG_ALL", "0.02"))
CUTOFF    = os.getenv("CUTOFF_DATE", "")        # e.g. "2025-06-01" for time-based split
TEST_SIZE = float(os.getenv("TEST_SIZE", "0.2"))# used if no CUTOFF
K_TOP     = int(os.getenv("TOPK", "10"))        # for top-K metrics
POS_TH    = float(os.getenv("POS_THRESHOLD", "4.0"))  # rating >= POS_TH is positive in test

In [ ]:
os.makedirs(ART_DIR, exist_ok=True)

In [27]:
# ==== 9) Build Surprise train/test
def build_train_test_from_ratings(ratings_df: pd.DataFrame, cutoff_date: str = "", test_size: float = 0.2):
    """Create Surprise trainset/testset either by time split or random split."""
    reader = Reader(rating_scale=(1, 5))
    if cutoff_date:
        T = pd.to_datetime(cutoff_date)
        train_df = ratings_df[ratings_df["latest_ts"] < T]
        test_df  = ratings_df[ratings_df["latest_ts"] >= T]
        if train_df.empty or test_df.empty:
            raise ValueError("Time-based split produced empty train or test; adjust CUTOFF_DATE.")
        train_data = Dataset.load_from_df(train_df[["user_id","yacht_id","rating"]], reader)
        trainset = train_data.build_full_trainset()
        testset  = list(test_df[["user_id","yacht_id","rating"]].itertuples(index=False, name=None))
        return trainset, testset, train_df, test_df
    else:
        data = Dataset.load_from_df(ratings_df[["user_id","yacht_id","rating"]], reader)
        trainset, testset = train_test_split(data, test_size=test_size, random_state=42)
        # For convenience, reconstruct DataFrames
        train_df = pd.DataFrame(trainset.build_testset(), columns=["user_id","yacht_id","rating"])
        test_df  = pd.DataFrame(testset, columns=["user_id","yacht_id","rating"])
        return trainset, testset, train_df, test_df

In [30]:
trainset, testset, train_df, test_df = build_train_test_from_ratings(ratings, CUTOFF, TEST_SIZE)
train_df = ensure_str_ids(train_df)
test_df = ensure_str_ids(test_df)
print(train_df.shape)
print(test_df.shape)

(1192, 3)
(299, 3)


In [31]:
# ==== 10 Train model
def make_algo(name: str):
    if name == "SVDpp":
        return SVDpp(n_factors=FACTORS, n_epochs=EPOCHS, random_state=42)
    # default SVD
    return SVD(n_factors=FACTORS, n_epochs=EPOCHS, lr_all=LR_ALL, reg_all=REG_ALL, random_state=42)

In [32]:
algo = make_algo(ALGO_NAME)
algo.fit(trainset)

In [33]:
# ==== 11 Evaluate: RMSE/MAE (pointwise)
preds = algo.test(testset)
rmse = accuracy.rmse(preds, verbose=False)
mae  = accuracy.mae(preds, verbose=False)
print(f"[METRIC] RMSE={rmse:.4f}  MAE={mae:.4f}")

[METRIC] RMSE=1.2529  MAE=0.8580


In [34]:
# ==== 12  Evaluate: Top-K ranking metrics
def precision_recall_ndcg_at_k(algo, train_df, test_df, K=10, pos_threshold=4.0):
    # positives in test
    test_pos = test_df[test_df["rating"] >= pos_threshold]
    user_pos = test_pos.groupby("user_id")["yacht_id"].apply(set).to_dict()
    # candidates = items seen in train universe
    item_set = set(train_df["yacht_id"].unique())
    # items seen in train per user
    train_seen = train_df.groupby("user_id")["yacht_id"].apply(set).to_dict()

    precs, recs, ndcgs = [], [], []

    for u, pos_items in user_pos.items():
        candidates = list(item_set - train_seen.get(u, set()))
        if not candidates:
            continue
        est = [(iid, algo.predict(u, iid).est) for iid in candidates]
        est.sort(key=lambda x: x[1], reverse=True)
        top = [iid for iid,_ in est[:K]]

        # precision/recall
        hits = len(set(top) & pos_items)
        precs.append(hits / K)
        recs.append(hits / max(1, len(pos_items)))

        # nDCG
        # relevance is 1 if item in pos_items else 0
        gains = [1.0 if iid in pos_items else 0.0 for iid in top]
        dcg = 0.0
        for i, g in enumerate(gains, start=1):
            dcg += g / np.log2(i + 1)
        # ideal DCG: first min(K, |pos|) are 1
        ideal_hits = min(K, len(pos_items))
        idcg = sum(1.0 / np.log2(i + 1) for i in range(1, ideal_hits + 1))
        ndcgs.append(dcg / idcg if idcg > 0 else 0.0)

    metrics = {
        "precision_at_k": float(np.mean(precs)) if precs else 0.0,
        "recall_at_k": float(np.mean(recs)) if recs else 0.0,
        "ndcg_at_k": float(np.mean(ndcgs)) if ndcgs else 0.0,
        "users_evaluated": int(len(precs)),
    }
    return metrics

In [35]:
rank_metrics = precision_recall_ndcg_at_k(algo, train_df, test_df, K=K_TOP, pos_threshold=POS_TH)
print(f"[METRIC] P@{K_TOP}={rank_metrics['precision_at_k']:.4f}  "
      f"R@{K_TOP}={rank_metrics['recall_at_k']:.4f}  "
      f"nDCG@{K_TOP}={rank_metrics['ndcg_at_k']:.4f}  "
      f"(users={rank_metrics['users_evaluated']})")

[METRIC] P@15=0.0000  R@15=0.0000  nDCG@15=0.0000  (users=34)


==== 13 Interpretation (“why recommended?”)
-----------------------------
For SVD/SVDpp we can use:
 - global/user/item biases (SVD): algo.bi (item bias), algo.bu (user bias)
 - cosine similarity to user's recently liked items (post-hoc explanation)

We'll build a tiny helper that, for a given (user, candidate),
shows top-3 "most similar liked items" by cosine in latent space.

In [36]:
def latent_vectors(algo):
    # item latent matrix (inner item id -> vector)
    Qi = algo.qi
    # mapping inner <-> raw ids
    inner_to_raw_iid = {i: trainset.to_raw_iid(i) for i in range(trainset.n_items)}
    raw_to_inner_iid = {raw: inner for inner, raw in inner_to_raw_iid.items()}
    return Qi, raw_to_inner_iid, inner_to_raw_iid

In [37]:
Qi, raw2inner, inner2raw = latent_vectors(algo)

In [38]:
def cosine(a, b, eps=1e-12):
    na = np.linalg.norm(a); nb = np.linalg.norm(b)
    if na < eps or nb < eps:
        return 0.0
    return float(np.dot(a, b) / (na * nb))

In [39]:
def explain_reco(user_id: str, candidate_iid: str, top_m=3):
    """Return explanation: similar previously liked items for this user."""
    # 1) map user to inner id
    try:
        u_in = trainset.to_inner_uid(user_id)
    except ValueError:
        return {"note": "user not in train"}

    # 2) items the user interacted with in TRAIN (inner ids)
    user_items_inner = [i for (i, _rating) in trainset.ur[u_in]]

    # 3) convert inner -> raw item ids
    user_items_raw = [trainset.to_raw_iid(i) for i in user_items_inner]

    # 4) pick user's positives (rating >= POS_TH) from TRAIN df
    liked_raw = set(
        train_df.loc[
            (train_df.user_id.astype(str) == str(user_id)) &
            (train_df.rating >= POS_TH),
            "yacht_id"
        ].astype(str)
    )

    liked = [iid for iid in user_items_raw if iid in liked_raw]
    if not liked:
        return {"note": "no liked items in train"}

    # 5) candidate latent vector
    if candidate_iid not in raw2inner:
        return {"note": "candidate not in train"}
    cand_vec = Qi[raw2inner[candidate_iid]]

    # 6) compute cosine similarity with liked items
    sims = []
    for iid in liked:
        if iid not in raw2inner:
            continue
        sims.append((iid, cosine(cand_vec, Qi[raw2inner[iid]])))

    sims.sort(key=lambda x: x[1], reverse=True)
    return {"similar_liked": sims[:top_m]}

In [ ]:
# ==== 14  Save artifacts & sample outputs
model_path = os.path.join(ART_DIR, f"surprise_{ALGO_NAME.lower()}.pkl")
meta_path  = os.path.join(ART_DIR, f"surprise_meta_{ALGO_NAME.lower()}.json")
metrics_path = os.path.join(ART_DIR, f"metrics_{ALGO_NAME.lower()}.json")
samples_path = os.path.join(ART_DIR, f"sample_recommendations_{ALGO_NAME.lower()}.csv")

In [ ]:
# ==== 15 Save model
with open(model_path, "wb") as f:
    pickle.dump(algo, f)

In [ ]:
# Save meta (train universe for inference & filtering)
meta = {
    "algo": ALGO_NAME,
    "trained_utc": datetime.now(timezone.utc).isoformat(),
    "params": {"factors": FACTORS, "epochs": EPOCHS, "lr_all": LR_ALL, "reg_all": REG_ALL},
    "num_users": int(train_df.user_id.nunique()),
    "num_items": int(train_df.yacht_id.nunique()),
    "users": sorted(str(uid) for uid in train_df.user_id.unique().tolist()),
    "items": sorted(str(iid) for iid in train_df.yacht_id.unique().tolist()),
}
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)

In [ ]:
# Save metrics
all_metrics = {
    "rmse": rmse,
    "mae": mae,
    "precision_at_k": rank_metrics["precision_at_k"],
    "recall_at_k": rank_metrics["recall_at_k"],
    "ndcg_at_k": rank_metrics["ndcg_at_k"],
    "users_evaluated": rank_metrics["users_evaluated"],
    "k": K_TOP,
    "pos_threshold": POS_TH,
    "cutoff_date": CUTOFF or None,
    "test_size": TEST_SIZE if not CUTOFF else None,
}
with open(metrics_path, "w") as f:
    json.dump(all_metrics, f, indent=2)

In [ ]:
print(f"[OK] Saved model → {model_path}")
print(f"[OK] Saved meta  → {meta_path}")
print(f"[OK] Saved metrics → {metrics_path}")

In [40]:
# ==== 16 Generate sample top-N for first N users (for QA / quick sanity check)
def recommend_for_user(user_id: str, limit=10):
    # candidate universe = items seen in train
    item_set = set(train_df["yacht_id"].unique())
    # filter out items seen by user in train to avoid echo
    try:
        seen = set(train_df[train_df.user_id == user_id]["yacht_id"].unique())
    except KeyError:
        seen = set()
    candidates = list(item_set - seen)
    if not candidates:
        return []
    est = [(iid, algo.predict(user_id, iid).est) for iid in candidates]
    est.sort(key=lambda x: x[1], reverse=True)
    return [iid for iid,_ in est[:limit]]

In [41]:
sample_users = train_df["user_id"].drop_duplicates().head(20).tolist()
rows = []
for u in sample_users:
    top10 = recommend_for_user(u, limit=10)
    # add a tiny explanation for the first item
    exp = explain_reco(u, top10[0]) if top10 else {}
    rows.append({
        "user_id": u,
        "top10": ",".join(top10),
        "explanation_first": json.dumps(exp)
    })
reco=pd.DataFrame(rows)
reco.head()


,user_id,top10,explanation_first
0,a51e935b-aead-4240-963a-a6d2c1d357fb,"bfd96438-e780-4de7-b302-ab36031de9cf,11801fa1-...","{""similar_liked"": [[""d5bad0f1-27fe-448f-9ed8-1..."
1,4111cbd5-9b7c-463c-8296-dcbfc7c61f8a,"5b549d30-6e16-4e78-b0f8-5a6992bd53c6,0d746270-...","{""similar_liked"": [[""c3fd7690-25ea-4697-a26a-3..."
2,35d7b622-eb0e-4cd2-bfe1-24a52fa0a3b8,"ba4e1f53-c624-47d8-af19-a2c9f278fff4,fdf36ea6-...","{""similar_liked"": [[""9504e930-818a-4f1b-85af-e..."
3,b4712c9a-1b82-4598-a178-572c7b14957e,"5b549d30-6e16-4e78-b0f8-5a6992bd53c6,6acfca25-...","{""note"": ""no liked items in train""}"
4,501716f9-9987-46dd-be64-4b3570d97cee,"e5f54931-d974-4436-8fab-a381a9a0ab44,d915adc0-...","{""note"": ""no liked items in train""}"


In [ ]:
reco.to_csv(samples_path, index=False)
print(f"[OK] Saved sample recommendations → {samples_path}")

-----------------------------
--- STORE RECOMMENDATIONS IN DB ---------
-----------------------------
==== Save per-user recommendations into users.recommendations (uuid[]) =========

In [42]:
CANDIDATES_N = 30
BUSINESS_LIMIT = 12
BUSINESS_STRATEGY = "high_price_then_sort_by_rating"

In [43]:
ITEM_UNIVERSE = set(train_df["yacht_id"].unique())
SEEN_BY_USER = train_df.groupby("user_id")["yacht_id"].apply(set).to_dict()

In [44]:
# ==== 17 Ensure column exists (uuid[])
with engine.begin() as conn:
    conn.execute(text("""
    ALTER TABLE users
    ADD COLUMN IF NOT EXISTS "recommendations" uuid[];
    """))

In [45]:
HYDRATE_SQL = text("""
  SELECT id, "summerLowSeasonPrice" AS price, rating
  FROM yachts
  WHERE id IN (SELECT UNNEST(CAST(:ids AS uuid[])))
""")

In [46]:
# We pass a Python list of (string) UUIDs, cast to uuid[] safely via UNNEST → array_agg
UPDATE_USER_RECS = text("""
  UPDATE users
  SET "recommendations" = :yacht_ids
  WHERE id = :user_id
""").bindparams(
    bindparam("yacht_ids", type_=ARRAY(UUID)),
    bindparam("user_id",   type_=UUID),
)

In [47]:
# ==== 18 create helpers
def recommend_ids_for_user(user_id: str, limit=CANDIDATES_N):
    """Top-N candidate yacht IDs for a user (excluding items seen in train)."""
    candidates = list(ITEM_UNIVERSE - SEEN_BY_USER.get(user_id, set()))
    if not candidates:
        return []
    est = [(iid, algo.predict(user_id, iid).est) for iid in candidates]
    est.sort(key=lambda x: x[1], reverse=True)
    return [iid for iid, _ in est[:limit]]

In [48]:
def hydrate_yachts(ids):
    """Fetch price/rating for given yacht IDs preserving the input order."""
    if not ids:
        return pd.DataFrame(columns=["id","price","rating"])
    df = pd.read_sql(HYDRATE_SQL, engine, params={"ids": ids})
    order = {yid: i for i, yid in enumerate(ids)}
    df["__ord"] = df["id"].map(order)
    return df.sort_values("__ord").drop(columns="__ord")

In [49]:
def business_select_and_sort(df_y):
    """Pick 12 highest-price, then sort for return by rating desc (tie: price desc)."""
    if df_y.empty:
        return []
    top_by_price = df_y.sort_values("price", ascending=False).head(BUSINESS_LIMIT)
    top_by_price["rating"] = pd.to_numeric(top_by_price["rating"], errors="coerce")
    top_by_price["__r"] = top_by_price["rating"].fillna(-1)
    final_sorted = top_by_price.sort_values(["__r","price"], ascending=[False, False]).drop(columns="__r")
    return final_sorted["id"].tolist()

In [50]:
# ==== 18 Generate & save for all users from train (you can filter to "active" users if needed)
all_users = sorted(train_df["user_id"].unique().tolist())
print(f"[INFO] Generating & saving recommendations into users.recommendations for {len(all_users)} users...")

[INFO] Generating & saving recommendations into users.recommendations for 101 users...


In [52]:
print(all_users)

['0176a5a3-e4de-4365-a131-b62f095d7e8a', '0670dada-5602-449f-95a2-92d24c2c750a', '069c4703-54f3-4d21-8ef3-d542ca1872ae', '0d25cda9-ffd9-411e-ac9a-79a6f444a4f8', '0e14d57d-bfb1-4523-91ef-989dbbb4cc76', '0eeb1f59-3484-4c4e-9cf6-42e5e877838f', '13bdccb1-5e27-423d-bb7a-c85fcca7d484', '1a28c2df-b208-437e-8512-3c1c8210f6ef', '1a4f43f7-52e8-456d-8a96-34b704ba7eb6', '1a92aaef-67df-432c-aa50-bcac443c5472', '1c63e1f1-8b23-4bc2-8792-c5281d0e1be8', '1d7dbee7-2c6c-4d6a-b36c-4dc50a14b120', '23f09fda-ccba-4081-a482-b3d4d3e2d217', '24e79705-1038-4d6f-b4ad-cda02fd6e3da', '258d3174-d971-4692-98d1-d3b762d0b74b', '29a07150-9e6c-47c4-bb81-ed100d104a0b', '2a8b07cc-bb88-4c28-92e7-b1fcf7bcaae7', '304d7f41-c837-43bc-af43-0c554cef88f1', '35d7b622-eb0e-4cd2-bfe1-24a52fa0a3b8', '38d4b4c9-351f-483c-8771-5320693561bb', '4111cbd5-9b7c-463c-8296-dcbfc7c61f8a', '4150a63e-ab37-4f8a-82ef-97aa373888ec', '4352660e-2f10-4b84-beb2-bcdfd55fb651', '44c2b365-9c8f-4957-9d11-262da0624b72', '46d3c254-347e-439c-b08e-9d772d67f3f9',

In [59]:

# test
for user_id in all_users[:5]:
    cand_ids = recommend_ids_for_user(user_id, limit=CANDIDATES_N)
    print("Recommendations:", cand_ids )
    print("------------------------------------")

Recommendations: ['dd32a817-168f-4c79-96bc-ed3245158b82', 'bfd96438-e780-4de7-b302-ab36031de9cf', 'c570a588-d0c7-4311-8855-cc46438c54c7', '5b549d30-6e16-4e78-b0f8-5a6992bd53c6', 'e5f54931-d974-4436-8fab-a381a9a0ab44', '908ffa86-06af-4635-8c0c-156ea5c604d1', '6acfca25-05d0-4d0b-b272-572cbc84ecaf', '7bfe55af-70c9-4e29-8e7e-f4886ce3e79f', 'aa7fc898-aeb1-4d87-b03e-bb009c583809', 'fe2b3170-8b9f-4bc4-b34f-c1aedff80f74', 'b94df5cf-fd6e-47c2-b98e-d8fed429de71', 'e1c7f87d-052d-4c83-a29a-92a74b28c6b4', '48d0683d-ae63-4d17-9db8-0c80e0805823', '57ceb956-87e6-4361-a7ce-a7bbedbe497f', '38d8a8e6-924a-48e4-81e0-ff896c97c521', '24414531-03be-4b47-8ebf-41c18f65f32b', 'c3fd7690-25ea-4697-a26a-30230d5fa69e', '49faaf3e-c277-4e3a-88d6-c32352c8d811', '880e1c0e-2cfe-455a-b279-4e7a080e4b96', 'f6a70b5b-2ffa-4cd7-b133-335e587fab8a', '215d8802-eacc-4fed-a850-db38c39a9ed1', 'e110d4e7-aa51-4e73-abd0-f2c3ec5e56f1', '7f1e326b-0ab3-4863-adeb-74f7764cc077', '40b018ea-342a-4d6c-8ec8-f33e4a3e2c17', 'd915adc0-d6a9-405a-a2

In [ ]:
processed = 0
for user_id in all_users:
    cand_ids = recommend_ids_for_user(user_id, limit=CANDIDATES_N)
    if not cand_ids:
        continue
    ydf = hydrate_yachts(cand_ids)
    final_ids = business_select_and_sort(ydf)  # list of yacht UUIDs (as strings)

    if not final_ids:
        # Optionally set NULL when nothing to recommend:
        with engine.begin() as conn:
            conn.execute(text('UPDATE users SET "recommendations" = NULL WHERE id = :user_id::uuid'),
                      {"user_id": user_id})
        continue

    # Write uuid[] array into users.recommendations
    with engine.begin() as conn:
        conn.execute(UPDATE_USER_RECS, {"user_id": user_id, "yacht_ids": final_ids})

    processed += 1
    if processed % 200 == 0:
        print(f"[INFO] processed {processed} users...")

In [ ]:
print(f"[OK] Saved recommendations for {processed} users into users.recommendations.")